In [1]:
import pandas as pd
import sqlite3
import csv


pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)


game_box_score = pd.read_csv('/Users/gavincleach/Desktop/nba-enhanced-stats/2012-18_teamBoxScore.csv')
player_box_score = pd.read_csv('/Users/gavincleach/Desktop/nba-enhanced-stats/2012-18_playerBoxScore.csv')
standings_box_score = pd.read_csv('/Users/gavincleach/Desktop/nba-enhanced-stats/2012-18_standings.csv')

with sqlite3.connect('nba.db') as conn:
    conn.execute('DROP TABLE IF EXISTS game_box_score')
    game_box_score.to_sql('game_box_score', conn, index = False)
    

with sqlite3.connect('nba.db') as conn:
    conn.execute('DROP TABLE IF EXISTS player_box_score')
    player_box_score.to_sql('player_box_score', conn, index = False)
    
    
with sqlite3.connect('nba.db') as conn:
    conn.execute('DROP TABLE IF EXISTS standings_box_score')
    standings_box_score.to_sql('standings_box_score', conn, index = False)
    
    

def run_query(q):
    with sqlite3.connect('nba.db') as conn:
        return pd.read_sql(q, conn)


def run_command(command):
    with sqlite3.connect('nba.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(command)
        


In [2]:
run_command("""
ALTER TABLE game_box_score
ADD COLUMN homeTeam TEXT
""")

run_command("""
ALTER TABLE game_box_score
ADD COLUMN awayTeam TEXT
""")



run_command("""
UPDATE game_box_score
SET homeTeam = 
        CASE
            WHEN teamLoc = 'Home' THEN teamAbbr
            ELSE opptAbbr
            END; 
""")

run_command("""
UPDATE game_box_score
SET awayTeam = 
        CASE
            WHEN teamLoc = 'Away' THEN teamAbbr
            ELSE opptAbbr
            END; 
""")


run_command("""
ALTER TABLE player_box_score
ADD COLUMN homeTeam TEXT
""")

run_command("""
ALTER TABLE player_box_score
ADD COLUMN awayTeam TEXT
""")


run_command("""
UPDATE player_box_score
SET homeTeam = 
        CASE
            WHEN teamLoc = 'Home' THEN teamAbbr
            ELSE opptAbbr
            END; 
""")

run_command("""
UPDATE player_box_score
SET awayTeam = 
        CASE
            WHEN teamLoc = 'Away' THEN teamAbbr
            ELSE opptAbbr
            END; 
""")


run_query('''
SELECT COUNT(gmId), strftime('%Y', gmDate) AS 'Year' FROM nba_games
GROUP BY 2
''')

run_query('''
SELECT COUNT(*) AS 'Games', strftime('%Y', gmDate) AS 'Year', teamAbbr FROM game_box_score
WHERE teamAbbr = 'POR'
GROUP BY Year

''')

,Games,Year,teamAbbr
0,29,2012,POR
1,85,2013,POR
2,83,2014,POR
3,84,2015,POR
4,82,2016,POR
5,82,2017,POR
6,47,2018,POR


We want to simplify these so we can have more reader friendly data. We can get rid of columns that can be deduced from other columns and that may be excessive for our purposes.

In [3]:
run_command("""
DROP TABLE IF EXISTS nba_players""")

run_command("""
DROP TABLE IF EXISTS nba_standings""")

run_command("""
DROP TABLE IF EXISTS nba_games""")




run_command("""
CREATE TABLE IF NOT EXISTS nba_standings (
teamDateId TEXT PRIMARY KEY, 
teamAbbr TEXT, 
stDate DATE, 
rank int, 
gameWon int, 
gameLost int, 
stk TEXT, 
gameBack float, 
homeWin int, 
homeLoss int, 
awayWin int, 
awayLoss int, 
ptsScore int, 
ptsAllow int
)
""")


run_command("""
INSERT OR IGNORE INTO nba_standings
SELECT 
    teamAbbr || stDate, 
    teamAbbr,
    stDate, 
    rank, 
    gameWon, 
    gameLost, 
    stk, 
    gameBack, 
    homeWin, 
    homeLoss, 
    awayWin, 
    awayLoss, 
    ptsScore, 
    ptsAllow
FROM standings_box_score
""")


In [4]:
run_command("""
CREATE TABLE IF NOT EXISTS nba_games (
gmId TEXT PRIMARY KEY,
gmDate DATE,
seasTyp TEXT,
teamAbbr TEXT,
teamLoc TEXT,
teamRslt TEXT,
teamDayOff int,
teamPTS int,
teamAST int,
teamTO int,
teamSTL int,
teamBLK int,
teamPF int,
teamFGA int,
teamFGM int,
teamFGper decimal,
team2PA int,
team2PM int,
team2Pper decimal,
team3PA int,
team3PM int,
team3Pper decimal,
teamFTA int,
teamFTM int,
teamFTper decimal,
teamORB int,
teamDRB int,
teamPTS1 int,
teamPTS2 int,
teamPTS3 int,
teamPTS4 int,
teamPTS5 int,
teamPTS6 int,
teamPTS7 int,
teamPTS8 int,
opptAbbr TEXT,
opptLoc TEXT,
opptRslt TEXT,
opptDayOff int,
opptPTS int,
opptAST int,
opptTO int,
opptSTL int,
opptBLK int,
opptPF int,
opptFGA int,
opptFGM int,
opptFGper decimal,
oppt2PA int,
oppt2PM int,
oppt2Pper decimal,
oppt3PA int,
oppt3PM int,
oppt3Pper decimal,
opptFTA int,
opptFTM int,
opptFTper decimal,
opptORB int,
opptDRB int,
opptPTS1 int,
opptPTS2 int,
opptPTS3 int,
opptPTS4 int,
opptPTS5 int,
opptPTS6 int,
opptPTS7 int,
opptPTS8 int
)
""")

run_command("""
INSERT OR IGNORE INTO nba_games
SELECT 
    homeTeam || gmDate || awayTeam,  
    gmDate, 
    seasTyp, 
    teamAbbr, 
    teamLoc, 
    teamRslt, 
    teamDayOff, 
    teamPTS, 
    teamAST, 
    teamTO, 
    teamSTL, 
    teamBLK, 
    teamPF, 
    teamFGA, 
    teamFGM, 
    [teamFG%], 
    team2PA, 
    team2PM, 
    [team2P%], 
    team3PA, 
    team3PM, 
    [team3P%], 
    teamFTA, 
    teamFTM, 
    [teamFT%], 
    teamORB, 
    teamDRB, 
    teamPTS1, 
    teamPTS2, 
    teamPTS3, 
    teamPTS4, 
    teamPTS5, 
    teamPTS6, 
    teamPTS7, 
    teamPTS8, 
    opptAbbr, 
    opptLoc, 
    opptRslt, 
    opptDayOff, 
    opptPTS, 
    opptAST, 
    opptTO, 
    opptSTL, 
    opptBLK, 
    opptPF, 
    opptFGA, 
    opptFGM, 
    [opptFG%], 
    oppt2PA, 
    oppt2PM, 
    [oppt2P%], 
    oppt3PA, 
    oppt3PM, 
    [oppt3P%], 
    opptFTA, 
    opptFTM, 
    [opptFT%], 
    opptORB, 
    opptDRB, 
    opptPTS1, 
    opptPTS2, 
    opptPTS3, 
    opptPTS4, 
    opptPTS5, 
    opptPTS6, 
    opptPTS7, 
    opptPTS8 
FROM game_box_score
""")


In [5]:
run_command("""
CREATE TABLE IF NOT EXISTS nba_players (
pGmId TEXT PRIMARY KEY, 
pId TEXT, 
gmId TEXT,  
playLNm TEXT, 
playFNm TEXT, 
teamAbbr TEXT,
playStat TEXT, 
playMin int, 
playPos TEXT, 
playHeight int, 
playWeight int, 
playBDate DATE, 
playPTS int, 
playAST int, 
playTO int, 
playSTL int, 
playBLK int, 
playPF int, 
playFGA int, 
playFGM int, 
playFGper decimal, 
play2PA int, 
play2PM int, 
play2Pper decimal, 
play3PA int, 
play3PM int, 
play3Pper decimal, 
playFTA int, 
playFTM int, 
playFTper decimal, 
playORB int, 
playDRB int
)
""")


run_command("""
INSERT OR IGNORE INTO nba_players
SELECT
    playDispNm || homeTeam || gmDate || awayTeam, 
    playDispNm, 
    homeTeam || gmDate || awayTeam, 
    playLNm, 
    playFNm, 
    teamAbbr, 
    playStat, 
    playMin, 
    playPos, 
    playHeight, 
    playWeight, 
    playBDate, 
    playPTS, 
    playAST, 
    playTO, 
    playSTL, 
    playBLK, 
    playPF, 
    playFGA, 
    playFGM, 
    [playFG%], 
    play2PA, 
    play2PM, 
    [play2P%], 
    play3PA, 
    play3PM, 
    [play3P%], 
    playFTA, 
    playFTM, 
    [playFT%], 
    playORB, 
    playDRB
FROM player_box_score
""")


In [6]:
run_query('''
SELECT * FROM nba_standings LIMIT 5
''')

,teamDateId,teamAbbr,stDate,rank,gameWon,gameLost,stk,gameBack,homeWin,homeLoss,awayWin,awayLoss,ptsScore,ptsAllow
0,ATL2012-10-30,ATL,2012-10-30,3,0,0,-,0.5,0,0,0,0,0,0
1,BKN2012-10-30,BKN,2012-10-30,3,0,0,-,0.5,0,0,0,0,0,0
2,BOS2012-10-30,BOS,2012-10-30,14,0,1,L1,1.0,0,0,0,1,107,120
3,CHA2012-10-30,CHA,2012-10-30,3,0,0,-,0.5,0,0,0,0,0,0
4,CHI2012-10-30,CHI,2012-10-30,3,0,0,-,0.5,0,0,0,0,0,0


In [7]:
run_query('''
SELECT MAX(playPTS) FROM nba_players

''')

,MAX(playPTS)
0,70


In [8]:
run_query('''
SELECT COUNT(gmId), strftime('%Y', gmDate) AS 'Year' FROM nba_games
GROUP BY 2
''')



,COUNT(gmId),Year
0,456,2012
1,1239,2013
2,1245,2014
3,1238,2015
4,1247,2016
5,1268,2017
6,686,2018


In [9]:
run_query('''
SELECT * FROM nba_games
LIMIT 10
''')

run_query('''
SELECT COUNT(*) AS 'Games', strftime('%Y', gmDate) AS 'Year', teamAbbr FROM nba_games
WHERE teamAbbr = 'POR'
GROUP BY Year

''')

,Games,Year,teamAbbr
0,14,2012,POR
1,44,2013,POR
2,40,2014,POR
3,46,2015,POR
4,39,2016,POR
5,39,2017,POR
6,24,2018,POR


We should be getting full amount of rows for nba_players and nba_games...

Let's see if we can find the missing pieces by starting small and loooking at all of the home games from one team. (We will pick the best team, POR)

To make this all more readable, I will create a new column in `nba_games` which will identify the season as either 2012-2013, 2013-2014, 2014-2015, 2015-2016, 2016-2017, or 2017-2018.

In [10]:
run_command("""
ALTER TABLE nba_games
ADD COLUMN season TEXT
""")

run_command("""
UPDATE nba_games
SET season = 
        CASE
            WHEN gmDate BETWEEN '2012-10-27' AND '2013-10-1' THEN '2012-2013'
            WHEN gmDate BETWEEN '2013-10-27' AND '2014-10-1' THEN '2013-2014'
            WHEN gmDate BETWEEN '2014-10-27' AND '2015-10-1' THEN '2014-2015'
            WHEN gmDate BETWEEN '2015-10-27' AND '2016-10-1' THEN '2015-2016'
            WHEN gmDate BETWEEN '2016-10-27' AND '2017-10-1' THEN '2016-2017'
            WHEN gmDate BETWEEN '2017-10-27' AND '2018-10-1' THEN '2017-2018'
            ELSE 'THIS SEASON DID NOT SHOW UP!'
            END; 
""")

run_query('''
SELECT * FROM nba_games
LIMIT 10
''')

run_query('''
SELECT COUNT(*) AS 'Games', season, teamAbbr FROM nba_games
WHERE teamAbbr = 'POR'
GROUP BY 2
''')



,Games,season,teamAbbr
0,41,2012-2013,POR
1,41,2013-2014,POR
2,41,2014-2015,POR
3,41,2015-2016,POR
4,41,2016-2017,POR
5,38,2017-2018,POR
6,3,THIS SEASON DID NOT SHOW UP!,POR


So! Good news, we found a problem... We are only getting half of the games... maybe this is happening because of how we are condensing `game_box_score`